In [1]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
import asyncio
import os

import requests
import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from typing import Literal, Union
from pydantic.config import ConfigDict

from src.context import context

from src.utilities import *
from src.bubble import *

import instructor



Retrieved company Darty : 1707313014508x102198350946437700
Retrieved project Darty_trustpilot : 1707329196900x870734705097005300


In [2]:
aspects_evol_df = get("Aspect Evolution")

In [10]:
df

,Modified Date,Created Date,Created By,Company,Grouped by,Max Rating,Mean Rating,Min Rating,Period,Project,Category,Date,Count of 1s,Count of 2s,Count of 3s,Count of 4s,Count of 5s,_id,SubCategory
10,2024-02-17 11:51:54.661000+00:00,2024-02-17 11:51:54.661000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,Category,5,5.000000,5,M,1707329196900x870734705097005300,1708015570991x523003420569127600,2023-03-31 00:00:00+00:00,0,0,0,0,1,1708170714661x143679099384227860,NaN
11,2024-02-17 11:51:54.667000+00:00,2024-02-17 11:51:54.667000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,Category,4,4.000000,4,M,1707329196900x870734705097005300,1708015553720x307677447830442240,2023-06-30 00:00:00+00:00,0,0,0,1,0,1708170714667x304003249950719940,NaN
12,2024-02-17 11:51:54.672000+00:00,2024-02-17 11:51:54.672000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,Category,1,1.000000,1,M,1707329196900x870734705097005300,1708015569716x598042261554836900,2023-07-31 00:00:00+00:00,1,0,0,0,0,1708170714672x528334235806553000,NaN
13,2024-02-17 11:51:54.681000+00:00,2024-02-17 11:51:54.681000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,Category,1,1.000000,1,M,1707329196900x870734705097005300,1708015570991x523003420569127600,2023-07-31 00:00:00+00:00,1,0,0,0,0,1708170714681x833147147999415700,NaN
14,2024-02-17 11:51:54.684000+00:00,2024-02-17 11:51:54.684000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,Category,1,1.000000,1,M,1707329196900x870734705097005300,1708015553720x307677447830442240,2023-08-31 00:00:00+00:00,1,0,0,0,0,1708170714684x711066889230007600,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,2024-02-17 11:51:57.559000+00:00,2024-02-17 11:51:57.559000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,SubCategory,1,1.000000,1,M,1707329196900x870734705097005300,NaN,2024-02-29 00:00:00+00:00,2,0,0,0,0,1708170717559x607760421087489400,1708015570395x458495376435047900
216,2024-02-17 11:51:57.562000+00:00,2024-02-17 11:51:57.562000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,SubCategory,5,3.000000,1,M,1707329196900x870734705097005300,NaN,2024-02-29 00:00:00+00:00,3,0,0,0,3,1708170717562x494369200219084300,1708015570687x524341896218553100
217,2024-02-17 11:51:57.642000+00:00,2024-02-17 11:51:57.642000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,SubCategory,5,2.777778,1,M,1707329196900x870734705097005300,NaN,2024-02-29 00:00:00+00:00,5,0,0,0,4,1708170717642x290440497403705150,1708015571499x209656851322028060
218,2024-02-17 11:51:57.652000+00:00,2024-02-17 11:51:57.651000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,SubCategory,1,1.000000,1,M,1707329196900x870734705097005300,NaN,2024-02-29 00:00:00+00:00,1,0,0,0,0,1708170717651x126759052577435580,1708015573399x329923182630295500


In [9]:
df = aspects_evol_df[aspects_evol_df["Period"] == 'M']
df[["Date", "Category", "Mean Rating"]]


,Date,Category,Mean Rating
10,2023-03-31 00:00:00+00:00,1708015570991x523003420569127600,5.000000
11,2023-06-30 00:00:00+00:00,1708015553720x307677447830442240,4.000000
12,2023-07-31 00:00:00+00:00,1708015569716x598042261554836900,1.000000
13,2023-07-31 00:00:00+00:00,1708015570991x523003420569127600,1.000000
14,2023-08-31 00:00:00+00:00,1708015553720x307677447830442240,1.000000
...,...,...,...
215,2024-02-29 00:00:00+00:00,NaN,1.000000
216,2024-02-29 00:00:00+00:00,NaN,3.000000
217,2024-02-29 00:00:00+00:00,NaN,2.777778
218,2024-02-29 00:00:00+00:00,NaN,1.000000


In [11]:
aspects_evol_df = get("Aspect Evolution")
aspects_evol_df

,Modified Date,Created Date,Created By,Company,Grouped by,Max Rating,Mean Rating,Min Rating,Period,Project,Category,Date,Count of 1s,Count of 2s,Count of 3s,Count of 4s,Count of 5s,_id,SubCategory
0,2024-02-17 11:51:54.243000+00:00,2024-02-17 11:51:54.243000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,Category,4,2.750000,1,Y,1707329196900x870734705097005300,1708015553720x307677447830442240,2023-12-31 00:00:00+00:00,1,1,0,2,0,1708170714243x388283336860933500,NaN
1,2024-02-17 11:51:54.248000+00:00,2024-02-17 11:51:54.248000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,Category,1,1.000000,1,Y,1707329196900x870734705097005300,1708015555422x783692141072623600,2023-12-31 00:00:00+00:00,1,0,0,0,0,1708170714248x180229393162605980,NaN
2,2024-02-17 11:51:54.253000+00:00,2024-02-17 11:51:54.253000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,Category,3,1.800000,1,Y,1707329196900x870734705097005300,1708015569716x598042261554836900,2023-12-31 00:00:00+00:00,2,2,1,0,0,1708170714253x938878456568073200,NaN
3,2024-02-17 11:51:54.257000+00:00,2024-02-17 11:51:54.257000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,Category,5,2.888889,1,Y,1707329196900x870734705097005300,1708015570991x523003420569127600,2023-12-31 00:00:00+00:00,8,2,0,0,8,1708170714257x498038144946764350,NaN
4,2024-02-17 11:51:54.261000+00:00,2024-02-17 11:51:54.261000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,Category,5,4.333333,1,Y,1707329196900x870734705097005300,1708015577252x623767797641778300,2023-12-31 00:00:00+00:00,1,0,1,0,7,1708170714261x989330378063111800,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,2024-02-17 11:51:59.714000+00:00,2024-02-17 11:51:59.714000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,SubCategory,5,5.000000,5,d,1707329196900x870734705097005300,NaN,2024-02-07 00:00:00+00:00,0,0,0,0,1,1708170719714x398904548409440450,1708015554529x444035970008549250
375,2024-02-17 11:51:59.718000+00:00,2024-02-17 11:51:59.718000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,SubCategory,5,5.000000,5,d,1707329196900x870734705097005300,NaN,2024-02-07 00:00:00+00:00,0,0,0,0,1,1708170719718x788308666905528200,1708015555741x283185418299470850
376,2024-02-17 11:51:59.721000+00:00,2024-02-17 11:51:59.721000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,SubCategory,1,1.000000,1,d,1707329196900x870734705097005300,NaN,2024-02-07 00:00:00+00:00,1,0,0,0,0,1708170719721x711428211815957000,1708015570395x458495376435047900
377,2024-02-17 11:51:59.725000+00:00,2024-02-17 11:51:59.725000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,SubCategory,1,1.000000,1,d,1707329196900x870734705097005300,NaN,2024-02-07 00:00:00+00:00,1,0,0,0,0,1708170719725x834794098354054300,1708015570687x524341896218553100


In [33]:
import pandas as pd

# Create your DataFrame
df = pd.DataFrame({'Date': ['2023-06-30 00:00:00+00:00', '2023-06-30 00:00:00+00:00',
                           '2023-07-31 00:00:00+00:00', '2023-07-31 00:00:00+00:00',
                           '2023-08-31 00:00:00+00:00'],
                   'Category': ['1708015570991x523003420569127600', '1708015553720x307677447830442240',
                               '1708015569716x598042261554836900', '1708015570991x523003420569127600',
                               '1708015553720x307677447830442240'],
                   'Count': [5.0, 4.0, 1.0, 1.0, 1.0]})

# Convert 'Date' to datetime format and set as index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index(['Date', 'Category'], inplace=True)

# Resample to monthly frequency and fill missing values with 0
result = df.unstack(fill_value=0).asfreq('M', fill_value=0)

count_df = result["Count"]
count_df

/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_49001/3055869590.py:17: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  result = df.unstack(fill_value=0).asfreq('M', fill_value=0)


Category,1708015553720x307677447830442240,1708015569716x598042261554836900,1708015570991x523003420569127600
Date,,,
2023-06-30 00:00:00+00:00,4.0,0.0,5.0
2023-07-31 00:00:00+00:00,0.0,1.0,1.0
2023-08-31 00:00:00+00:00,1.0,0.0,0.0


In [31]:
count_df["1708015553720x307677447830442240"]

Date
2023-06-30 00:00:00+00:00    4.0
2023-07-31 00:00:00+00:00    0.0
2023-08-31 00:00:00+00:00    1.0
Freq: ME, Name: 1708015553720x307677447830442240, dtype: float64

In [35]:
df = aspects_evol_df

In [37]:
import pandas as pd

# Create your DataFrame
#df = pd.DataFrame({'Date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-02-01', '2023-02-02', '2023-02-03', '2023-03-01'],
#                   'Category': ['A', 'B', 'A', 'C', 'A', 'B', 'A']})

# Convert 'Date' to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Resample to monthly frequency and count occurrences of each category
result = df.resample('M', on='Date')['Category'].value_counts().unstack(fill_value=0)

# Print the result
result


/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_49001/237673877.py:11: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  result = df.resample('M', on='Date')['Category'].value_counts().unstack(fill_value=0)


Category,1708015553720x307677447830442240,1708015555422x783692141072623600,1708015569716x598042261554836900,1708015570991x523003420569127600,1708015577252x623767797641778300
Date,,,,,
2023-03-31 00:00:00+00:00,0,0,0,2,0
2023-06-30 00:00:00+00:00,2,0,0,0,0
2023-07-31 00:00:00+00:00,0,0,2,2,0
2023-08-31 00:00:00+00:00,2,0,0,2,0
2023-10-31 00:00:00+00:00,0,0,0,2,0
2023-11-30 00:00:00+00:00,2,0,0,2,0
2023-12-31 00:00:00+00:00,3,3,5,10,7
2024-01-31 00:00:00+00:00,20,12,15,24,8
2024-02-29 00:00:00+00:00,5,3,6,7,0


In [38]:
aspects_evol_df

,Modified Date,Created Date,Created By,Company,Grouped by,Max Rating,Mean Rating,Min Rating,Period,Project,Category,Date,Count of 1s,Count of 2s,Count of 3s,Count of 4s,Count of 5s,_id,SubCategory
0,2024-02-17 11:51:54.243000+00:00,2024-02-17 11:51:54.243000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,Category,4,2.750000,1,Y,1707329196900x870734705097005300,1708015553720x307677447830442240,2023-12-31 00:00:00+00:00,1,1,0,2,0,1708170714243x388283336860933500,NaN
1,2024-02-17 11:51:54.248000+00:00,2024-02-17 11:51:54.248000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,Category,1,1.000000,1,Y,1707329196900x870734705097005300,1708015555422x783692141072623600,2023-12-31 00:00:00+00:00,1,0,0,0,0,1708170714248x180229393162605980,NaN
2,2024-02-17 11:51:54.253000+00:00,2024-02-17 11:51:54.253000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,Category,3,1.800000,1,Y,1707329196900x870734705097005300,1708015569716x598042261554836900,2023-12-31 00:00:00+00:00,2,2,1,0,0,1708170714253x938878456568073200,NaN
3,2024-02-17 11:51:54.257000+00:00,2024-02-17 11:51:54.257000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,Category,5,2.888889,1,Y,1707329196900x870734705097005300,1708015570991x523003420569127600,2023-12-31 00:00:00+00:00,8,2,0,0,8,1708170714257x498038144946764350,NaN
4,2024-02-17 11:51:54.261000+00:00,2024-02-17 11:51:54.261000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,Category,5,4.333333,1,Y,1707329196900x870734705097005300,1708015577252x623767797641778300,2023-12-31 00:00:00+00:00,1,0,1,0,7,1708170714261x989330378063111800,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,2024-02-17 11:51:59.714000+00:00,2024-02-17 11:51:59.714000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,SubCategory,5,5.000000,5,d,1707329196900x870734705097005300,NaN,2024-02-07 00:00:00+00:00,0,0,0,0,1,1708170719714x398904548409440450,1708015554529x444035970008549250
375,2024-02-17 11:51:59.718000+00:00,2024-02-17 11:51:59.718000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,SubCategory,5,5.000000,5,d,1707329196900x870734705097005300,NaN,2024-02-07 00:00:00+00:00,0,0,0,0,1,1708170719718x788308666905528200,1708015555741x283185418299470850
376,2024-02-17 11:51:59.721000+00:00,2024-02-17 11:51:59.721000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,SubCategory,1,1.000000,1,d,1707329196900x870734705097005300,NaN,2024-02-07 00:00:00+00:00,1,0,0,0,0,1708170719721x711428211815957000,1708015570395x458495376435047900
377,2024-02-17 11:51:59.725000+00:00,2024-02-17 11:51:59.725000+00:00,admin_user_feedback-analysis_test,1707313014508x102198350946437700,SubCategory,1,1.000000,1,d,1707329196900x870734705097005300,NaN,2024-02-07 00:00:00+00:00,1,0,0,0,0,1708170719725x834794098354054300,1708015570687x524341896218553100


In [41]:
import pandas as pd

# Your DataFrame
df = pd.DataFrame({
    'Date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-02-01', '2023-02-02', '2023-02-03', '2023-03-01'],
    '_id': [1, 2, 3, 4, 5, 6, 7],
    'Category': ['A', 'B', 'A', 'C', 'A', 'B', 'A'],
    'SubCategory': ['X', 'Y', 'X', 'Z', 'X', 'Y', 'X']
})

# Convert 'Date' to datetime format
df['Date'] = pd.to_datetime(df['Date'])


In [52]:
df = aspects_evol_df[['Date', 'Category', 'SubCategory']]
df.fillna('', inplace=True)

df

/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_49001/4168413704.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna('', inplace=True)


,Date,Category,SubCategory
0,2023-12-31 00:00:00+00:00,1708015553720x307677447830442240,
1,2023-12-31 00:00:00+00:00,1708015555422x783692141072623600,
2,2023-12-31 00:00:00+00:00,1708015569716x598042261554836900,
3,2023-12-31 00:00:00+00:00,1708015570991x523003420569127600,
4,2023-12-31 00:00:00+00:00,1708015577252x623767797641778300,
...,...,...,...
374,2024-02-07 00:00:00+00:00,,1708015554529x444035970008549250
375,2024-02-07 00:00:00+00:00,,1708015555741x283185418299470850
376,2024-02-07 00:00:00+00:00,,1708015570395x458495376435047900
377,2024-02-07 00:00:00+00:00,,1708015570687x524341896218553100


In [53]:
result = df.resample('M', on='Date')[['Category', 'SubCategory']].value_counts().unstack(fill_value=0)
result

/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_49001/1307452999.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  result = df.resample('M', on='Date')[['Category', 'SubCategory']].value_counts().unstack(fill_value=0)


SubCategory                                                     \
Date                      Category                               
2023-03-31 00:00:00+00:00                                    0   
                          1708015570991x523003420569127600   2   
2023-06-30 00:00:00+00:00                                    0   
                          1708015553720x307677447830442240   2   
2023-07-31 00:00:00+00:00                                    0   
                          1708015569716x598042261554836900   2   
                          1708015570991x523003420569127600   2   
2023-08-31 00:00:00+00:00                                    0   
                          1708015553720x307677447830442240   2   
                          1708015570991x523003420569127600   2   
2023-10-31 00:00:00+00:00                                    0   
                          1708015570991x523003420569127600   2   
2023-11-30 00:00:00+00:00                                    0   
                          1708015553720x307677447830442240   2   
                          1708015570991x523003420569127600   2   
2023-12-31 00:00:00+00:00                                    0   
                          1708015553720x307677447830442240   3   
                          1708015555422x783692141072623600   3   
                          1708015569716x598042261554836900   5   
                          1708015570991x523003420569127600  10   
                          1708015577252x623767797641778300   7   
2024-01-31 00:00:00+00:00                                    0   
                          1708015553720x307677447830442240  20   
                          1708015555422x783692141072623600  12   
                          1708015569716x598042261554836900  15   
                          1708015570991x523003420569127600  24   
                          1708015577252x623767797641778300   8   
2024-02-29 00:00:00+00:00                                    0   
                          1708015553720x307677447830442240   5   
                          1708015555422x783692141072623600   3   
                          1708015569716x598042261554836900   6   
                          1708015570991x523003420569127600   7   
2024-12-31 00:00:00+00:00                                    0   
                          1708015553720x307677447830442240   1   
                          1708015555422x783692141072623600   1   
                          1708015569716x598042261554836900   1   
                          1708015570991x523003420569127600   1   
                          1708015577252x623767797641778300   1   

SubCategory                                                 1708015554131x995344137291493100  \
Date                      Category                                                             
2023-03-31 00:00:00+00:00                                                                  0   
                          1708015570991x523003420569127600                                 0   
2023-06-30 00:00:00+00:00                                                                  2   
                          1708015553720x307677447830442240                                 0   
2023-07-31 00:00:00+00:00                                                                  0   
                          1708015569716x598042261554836900                                 0   
                          1708015570991x523003420569127600                                 0   
2023-08-31 00:00:00+00:00                                                                  0   
                          1708015553720x307677447830442240                                 0   
                          1708015570991x523003420569127600                                 0   
2023-10-31 00:00:00+00:00                                                                  0   
                          1708015570991x523003420569127600                                 0   
2023-11-30 00:0

In [ ]:
row_label = 'your_desired_row_label'
category = df.loc[row_label, 'Category']
print(category)

In [64]:
i, row = next(result.iterrows())
i

(Timestamp('2023-03-31 00:00:00+0000', tz='UTC'), '')

In [58]:
result.iloc[1]

SubCategory
                                    2
1708015554131x995344137291493100    0
1708015554529x444035970008549250    0
1708015555741x283185418299470850    0
1708015568900x728837610303776400    0
1708015569398x419300495562577400    0
1708015570097x862705649430030700    0
1708015570395x458495376435047900    0
1708015570687x524341896218553100    0
1708015571499x209656851322028060    0
1708015573399x329923182630295500    0
1708015576620x263266380365956300    0
1708015577555x688807197740501000    0
1708015577923x774632487944991200    0
1708015580113x761173827728466800    0
Name: (2023-03-31 00:00:00+00:00, 1708015570991x523003420569127600), dtype: int64

In [56]:
result['1708015554131x995344137291493100']

Date                       Category                        
2023-03-31 00:00:00+00:00                                      0
                           1708015570991x523003420569127600    0
2023-06-30 00:00:00+00:00                                      2
                           1708015553720x307677447830442240    0
2023-07-31 00:00:00+00:00                                      0
                           1708015569716x598042261554836900    0
                           1708015570991x523003420569127600    0
2023-08-31 00:00:00+00:00                                      0
                           1708015553720x307677447830442240    0
                           1708015570991x523003420569127600    0
2023-10-31 00:00:00+00:00                                      0
                           1708015570991x523003420569127600    0
2023-11-30 00:00:00+00:00                                      0
                           1708015553720x307677447830442240    0
                           170

In [ ]:

# Convert 'Date' to datetime format
df['Date'] = pd.to_datetime(df['Date'])


In [ ]:
[['Category', 'SubCategory']]

In [45]:
result = df.resample('M', on='Date')[['Category', 'SubCategory']].value_counts().unstack(fill_value=0)
result

/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_49001/1307452999.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  result = df.resample('M', on='Date')[['Category', 'SubCategory']].value_counts().unstack(fill_value=0)


SubCategory          X  Y  Z
Date       Category         
2023-01-31 A         2  0  0
           B         0  1  0
2023-02-28 A         1  0  0
           B         0  1  0
           C         0  0  1
2023-03-31 A         1  0  0

In [18]:
_,row = next(df.iterrows())


4

In [19]:
for i, row in df.iterrows():
    bubble_client.update_object(bubble_type="Aspect Evolution", bubble_id=row["_id"], fields={"Count": sum([row["Count of "+str(k)+"s"] for k in range(1,6)])})
